<a href="https://colab.research.google.com/github/trvoid/llm-study/blob/main/bert/getting_started_with_distilbert_for_qa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DistilBERT for MLM 시작하기 (한국어)

**노트: distilbert-base-multilingual-cased를 기본 모델로 사용하였음**

이 실습은 아래 문서의 내용을 토대로 진행하였습니다.

* [2. 마스크 언어 모델(Masked Language Model) 미세조정](https://wikidocs.net/166833), Transformers (신경망 언어모델 라이브러리) 강좌
* [Fine-tuning a masked language model](https://huggingface.co/learn/llm-course/en/chapter7/3), LLM Course in Hugging Face

사용할 모델과 데이터셋은 다음과 같습니다.

* [DistilBERT](distilbert/distilbert-base-multilingual-cased):기본 모델 (단어에 대한 단순 임베딩이 아니라 맥락을 고려한 임베딩 수행)
* [DistilBertForMaskedLM](https://huggingface.co/docs/transformers/model_doc/distilbert#transformers.DistilBertForMaskedLM): DistilBERT 모델에 마스크 언어 모델 층을 추가한 것으로서 마스크 언어 모델 미세조정 훈련을 위한 모델
* [nsmc](https://huggingface.co/datasets/Blpeng/nsmc): 네이버 영화 리뷰 데이터

## 토크나이저 및 모델 적재

In [1]:
import torch

# GPU 사용 가능 여부 확인 및 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")

device: cuda


In [2]:
from transformers import AutoTokenizer, DistilBertForMaskedLM

model_checkpoint = "distilbert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = DistilBertForMaskedLM.from_pretrained(model_checkpoint).to(device)

2025-04-09 14:47:22.392622: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-09 14:47:22.400811: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744177642.409334 2911333 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744177642.411909 2911333 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744177642.419823 2911333 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [3]:
print(f"vocab size: {len(tokenizer)}")
print(f"model device: {next(model.parameters()).device}")
distilbert_num_parameters = model.num_parameters() / 1_000_000
print(f"DistilBERT number of parameters: {round(distilbert_num_parameters)}M")
print(f"BERT number of parameters: 110M")

vocab size: 119547
model device: cuda:0
DistilBERT number of parameters: 135M
BERT number of parameters: 110M


In [4]:
#text = "달콤한 꿀과 톡쏘는 칠리의 만남."
text = "달콤한 꿀과 톡쏘는 [MASK]의 만남."

inputs = tokenizer(text, return_tensors="pt", max_length=14, padding="max_length")
print(f"input_ids: {inputs["input_ids"][0]}")
print(f"len(input_ids): {len(inputs["input_ids"][0])}")

token_strs = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
print(f"token_strs: {token_strs}")

decoded_text = tokenizer.decode(inputs["input_ids"][0])
print(f"decoded_text: {decoded_text}")

input_ids: tensor([   101,   9061, 119309,  11102,   8964,  11882,   9874, 119099,  11018,
           103,   9637,   9248,  37004,    119,    102])
len(input_ids): 15
token_strs: ['[CLS]', '달', '##콤', '##한', '꿀', '##과', '톡', '##쏘', '##는', '[MASK]', '의', '만', '##남', '.', '[SEP]']
decoded_text: [CLS] 달콤한 꿀과 톡쏘는 [MASK] 의 만남. [SEP]


In [5]:
import torch

def find_topk_for_masked(tokenizer, model, text, topk=5):
    inputs = tokenizer(text, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items() if isinstance(v, torch.Tensor)}

    token_logits = model(**inputs).logits
    #print(token_logits.shape)
    
    # [MASK]의 위치를 찾고, 해당 logits을 추출합니다.
    #print(torch.where(inputs["input_ids"] == tokenizer.mask_token_id))
    mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
    #print(mask_token_index)
    mask_token_logits = token_logits[0, mask_token_index, :]
    #print(mask_token_logits)
    
    # 가장 큰 logits값을 가지는 [MASK] 후보를 선택합니다.
    top_5_tokens = torch.topk(mask_token_logits, topk, dim=1).indices[0].tolist()
    
    return top_5_tokens

topk_tokens = find_topk_for_masked(tokenizer, model, text, 5)
for token in topk_tokens:
    print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")

'>>> 달콤한 꿀과 톡쏘는 단의 만남.'
'>>> 달콤한 꿀과 톡쏘는 수의 만남.'
'>>> 달콤한 꿀과 톡쏘는 이의 만남.'
'>>> 달콤한 꿀과 톡쏘는 유의 만남.'
'>>> 달콤한 꿀과 톡쏘는 불의 만남.'


## 데이터셋 적재

In [6]:
from datasets import load_dataset

dataset_name = "nsmc"
nsmc_dataset = load_dataset(dataset_name, trust_remote_code=True)
print(nsmc_dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})


In [7]:
sample = nsmc_dataset["train"].shuffle(seed=42).select(range(3))

for row in sample:
    print(f"\n'>>> Review: {row['document']}'")
    print(f"'>>> Label: {row['label']}'")


'>>> Review: For Carl.칼 세이건으로 시작해서 칼 세이건으로 끝난다.'
'>>> Label: 1'

'>>> Review: 모든 면에서 너무 좋고, 특히 동양적이고'
'>>> Label: 1'

'>>> Review: 달콤한 꿀과 톡쏘는 칠리의 만남'
'>>> Label: 1'


## 훈련 데이터 준비

In [8]:
def tokenize_function(examples):
    result = tokenizer(examples["document"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

In [9]:
test_examples = {
    "document": ["그것은 훌륭한 영화였습니다.", "저는 그 영화를 추천하지 않습니다."]
}
test_result = tokenize_function(test_examples)
print(test_result)
token_strs = tokenizer.convert_ids_to_tokens(test_result["input_ids"][1])
print(token_strs)

{'input_ids': [[101, 8924, 118627, 10892, 10005, 118901, 11102, 42428, 119147, 119081, 48345, 119, 102], [101, 9663, 11018, 8924, 42428, 11513, 9765, 38631, 23665, 9523, 119081, 48345, 119, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'word_ids': [[None, 0, 0, 0, 1, 1, 1, 2, 2, 2, 2, 3, None], [None, 0, 0, 1, 2, 2, 3, 3, 3, 4, 4, 4, 5, None]]}
['[CLS]', '저', '##는', '그', '영화', '##를', '추', '##천', '##하지', '않', '##습', '##니다', '.', '[SEP]']


In [10]:
# 빠른 멀티스레딩을 작동시키기 위해서, batched=True를 지정합니다.
tokenized_datasets = nsmc_dataset.map(
    tokenize_function, batched=True, remove_columns=["id", "document", "label"]
)
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 50000
    })
})


In [11]:
print(f"model_max_length: {tokenizer.model_max_length}")

model_max_length: 512


In [12]:
#chunk_size = 128
chunk_size = 150

In [13]:
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_datasets["train"][:3]

for idx, sample in enumerate(tokenized_samples["input_ids"]):
    print(f"Review {idx} length: {len(sample)}")

Review 0 length: 17
Review 1 length: 31
Review 2 length: 3


In [14]:
concatenated_examples = {
    k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys()
}
total_length = len(concatenated_examples["input_ids"])
print(f"Concatenated reviews length: {total_length}")

Concatenated reviews length: 51


In [15]:
chunks = {
    k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_examples.items()
}

for chunk in chunks["input_ids"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 51'


In [16]:
def group_texts(examples):
    # 모든 텍스트들을 결합한다.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # 결합된 텍스트들에 대한 길이를 구한다.
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # `chunk_size`보다 작은 경우 마지막 청크를 삭제
    total_length = (total_length // chunk_size) * chunk_size
    # max_len 길이를 가지는 chunk 단위로 슬라이스
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # 새로운 레이블 컬럼을 생성
    result["labels"] = result["input_ids"].copy()
    return result                            

In [17]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 25855
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 8643
    })
})

In [18]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

'##도 아까움. [SEP] [CLS] 원작의 긴장감을 제대로 살려내지못했다. [SEP] [CLS] 별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지.. 정말 발로해도 그것보단 낫겟다 납치. 감금만반복반복.. 이드라마는 가족도없다 연기못하는사람만모엿네 [SEP] [CLS] 액션이 없는데도 재미 있는 몇안되는 영화 [SEP] [CLS] 왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나? [SEP]'

In [19]:
tokenizer.decode(lm_datasets["train"][1]["labels"])

'##도 아까움. [SEP] [CLS] 원작의 긴장감을 제대로 살려내지못했다. [SEP] [CLS] 별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지.. 정말 발로해도 그것보단 낫겟다 납치. 감금만반복반복.. 이드라마는 가족도없다 연기못하는사람만모엿네 [SEP] [CLS] 액션이 없는데도 재미 있는 몇안되는 영화 [SEP] [CLS] 왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나? [SEP]'

In [20]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [21]:
samples = [lm_datasets["train"][i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] 아 더빙.. 진짜 [MASK] [MASK]나네요 [MASK]소리 [SEP] [CLS] [UNK]... 포스터보고 초딩영 [MASK]줄.. [MASK] [MASK] 오버연기 [MASK]차Ժ볍 [MASK] 않구나 [SEP] [CLS] [UNK] [SEP] [CLS] 교도소 이야기구먼.. 솔직히 재미는 [MASK] [MASK]. 평점 조정 [SEP] [CLS] 사이몬 [MASK]그 [MASK] 익살 PS런 [MASK]기가 돋보였던 영화! 스파이더맨에서 늙어보이기만 했던 커 [MASK]틴 던스트가 너무나 [MASK] 이뻐보였다 [SEP] [CLS] 막 걸음마 [UNK] 3 [MASK]부터 [MASK]등학교 1학년생인 8살용粘화 [MASK] [UNK]. [MASK]. 별반개'

'>>> ##도 아까움. [SEP] [CLS] 원 [MASK]의 agricultural장감을 제대로 살려내지 [MASK]했다. [SEP] [CLS] 별 [MASK]개도 아깝다 [MASK]나온다 이응경 길용우 연기생활이몇년 [MASK] [MASK].. 정말 발로 [MASK]도 [MASK]것보단 낫겟다 납치. 감 [MASK]만반복반복.. 이드라마 [MASK] 가족도없다 연기못하는사람만모엿네 [SEP] [CLS] 액 [MASK]이 없는데 [MASK] 재 [MASK] 있는 [MASK]안되는 영화 [SEP] [CLS] 왜케 평점 [MASK] 낮은건데? 꽤 볼만한데.. 헐리우 [MASK] [MASK] 화 [MASK]함에만 너무 길들여져 있나? [SEP]'


In [22]:
import collections
import numpy as np

from transformers import default_data_collator

wwm_probability = 0.2

def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # 단어와 해당 토큰 인덱스 간의 map 생성
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # 무작위로 단어 마스킹
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id

    return default_data_collator(features)

In [23]:
samples = [lm_datasets["train"][i] for i in range(2)]
batch = whole_word_masking_data_collator(samples)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] 아 더빙 [MASK]. [MASK] [MASK] 짜증나네요 목소리 [SEP] [CLS] [UNK]... 포스터보고 [MASK] [MASK] [MASK] [MASK] [MASK] [MASK]. [MASK]. 오버연기조차 가볍지 않구나 [SEP] [CLS] [UNK] [SEP] [CLS] 교도소 이야기구먼 [MASK]. 솔직히 [MASK] [MASK] [MASK] [MASK].. 평점 [MASK] [MASK] [SEP] [CLS] 사이몬페그의 익살스런 연기가 [MASK] [MASK] [MASK] 영화! 스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다 [SEP] [CLS] 막 걸음마 [UNK] 3세부터 [MASK] [MASK] 1학년생인 8살용영화. [UNK]... 별반개'

'>>> ##도 아까움. [SEP] [CLS] [MASK] [MASK] [MASK] 긴장감을 제대로 [MASK] [MASK] [MASK] [MASK] [MASK] [MASK]. [SEP] [CLS] [MASK] 반개도 아깝다 욕나온다 이응경 길용우 [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK]. [MASK] 정말 [MASK] [MASK] [MASK] [MASK] 그것보단 낫겟다 납치. 감금만반복반복.. 이드라마는 가족도없다 연기못하는사람만모엿네 [SEP] [CLS] 액션이 없는데도 재미 [MASK] 몇안되는 영화 [SEP] [CLS] [MASK] [MASK] 평점이 낮은건데? [MASK] 볼만한데.. [MASK] [MASK] [MASK] [MASK] [MASK] 화려함에만 너무 [MASK] [MASK] [MASK] [MASK] 있나? [SEP]'


훈련 데이터 개수와 테스트 데이터 개수를 지정하여 샘플링합니다.

In [24]:
#train_size = 10_000
#test_size = int(0.1 * train_size)
train_size = None
test_size = 0.1

downsampled_dataset = lm_datasets["train"].train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 23269
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 2586
    })
})

## 미세조정 훈련

In [25]:
from transformers import TrainingArguments, Trainer

batch_size = 64
# Show the training loss with every epoch
logging_steps = len(downsampled_dataset["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

training_args = TrainingArguments(
    output_dir=f"{model_name}-finetuned-nsmc",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=False,
    fp16=True,
    logging_steps=logging_steps,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=downsampled_dataset["train"],
    eval_dataset=downsampled_dataset["test"],
    data_collator=data_collator,
)

/home/wjeong/DevEnv/py312/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [26]:
import math

eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

>>> Perplexity: 274.00


In [27]:
trainer.train()

finetuned_model_path = "./fine-tuned-distilbert-nsmc-mlm"
tokenizer.save_pretrained(finetuned_model_path)
model.save_pretrained(finetuned_model_path)

Epoch,Training Loss,Validation Loss,Model Preparation Time
1,3.604100,3.070170,0.000500
2,3.107300,2.869706,0.000500
3,2.981400,2.876446,0.000500


In [28]:
eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

>>> Perplexity: 17.04


In [29]:
print(f"'input text: {text}'")
topk_tokens = find_topk_for_masked(tokenizer, model, text, topk=5)
for token in topk_tokens:
    print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")

'input text: 달콤한 꿀과 톡쏘는 [MASK]의 만남.'
'>>> 달콤한 꿀과 톡쏘는 영화의 만남.'
'>>> 달콤한 꿀과 톡쏘는 맛의 만남.'
'>>> 달콤한 꿀과 톡쏘는 것의 만남.'
'>>> 달콤한 꿀과 톡쏘는 잔의 만남.'
'>>> 달콤한 꿀과 톡쏘는 한의 만남.'
